In [ ]:
pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

··········


In [ ]:
system_prompt =  """مهمتك ان تساعد في الاجابة على اسئلة. سوف تعطى سؤال وسياق من ١٠ قطع مستخلصة من مستند طويل ومرتبة حسب التشابه مع السؤال. يجب عليك ان تجيب باختصار شديد على فقط اذا كان الجواب موجود في السياق. لا تجب على السؤال في حال لم يكن الجواب موجود في السياق، انما اجب "لا ادري" فلا تخترع اجوبة. اجب فقط بالعربية"""

# system_prompt = """
# You are an intelligent research assistant that answers questions in Arabic based on information from Arabic texts. Your primary goal is accuracy, and you should never fabricate information or make assumptions beyond the provided context.

# Task:

# 1. Carefully read the Arabic question and understand the intent.
# 2. Thoroughly analyze the provided Arabic passages (text chunks).
# 3. If the answer is explicitly stated or logically inferred from the passages, provide an accurate and concise answer in Arabic.
# 4. If there is insufficient information in the passages to answer the question definitively, respond with "لا يمكنني الإجابة بناءً على المعلومات المتوفرة.($$)" (I cannot answer based on the available information.)

# Output the answer after "Answer:". Then any additional info after "Note:"
# STRICTLY answer in Arabic only and Never speculate, make assumptions, or generate information that is not directly supported by the given passages.
# """

In [ ]:
import pandas as pd
from sentence_transformers import util
import replicate

# Function to calculate cosine similarities and retrieve sentences
def retrieve_top_sentences(question_embedding, all_sentences_embeddings, gold_idx, k=20):
    similarities = util.cos_sim(question_embedding, all_sentences_embeddings)
    rank = (similarities[0] >= similarities[0, gold_idx]).sum()
    top_k_indices = similarities[0].argsort(descending=True)[:k]
    return df_documents.iloc[top_k_indices], rank.item()

def format_prompt(question, context):
  PROMPT = f"السؤال:\n{question}\nالسياق:\n"
  for i in range(len(context)):
    PROMPT+= f"{i+1}- {context[i]}\n"
  return PROMPT

def get_answer(model, prompt):
    attempts = 0
    max_attempts = 3

    while attempts < max_attempts:
        try:
            response = replicate.run(
                model,
                input={
                    "prompt": prompt,
                    "system_prompt": system_prompt,
                },
            )
            return "".join(response)
            # answer = "".join(response)
            # pattern = r"Answer:(.*?)Note:"
            # match = re.search(pattern, answer, re.S)
            # return match.group(1).strip()  # Assuming response is a list of strings that needs to be joined
        except Exception as e:
            print(f"Attempt {attempts + 1} failed: {e}")
            attempts += 1
            if attempts == max_attempts:
                return "NA"


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import numpy as np
import ast
from tqdm import tqdm


# Load data
df_questions = pd.read_csv("cleaned_dataset.csv")
df_documents = pd.read_csv("cleaned_document_dataset.csv")

def str_to_array(s):
    # Using ast.literal_eval to safely evaluate the string
    return np.array(ast.literal_eval(s))

df_questions['question_formal_embedding'] = df_questions['question_formal_embedding'].apply(str_to_array)
df_questions['question_conversational_embedding'] = df_questions['question_conversational_embedding'].apply(str_to_array)
df_documents['embedding'] = df_documents['embedding'].apply(str_to_array)

# Iterate over questions
for idx, row in tqdm(df_questions.iterrows(), total=df_questions.shape[0], desc="Processing Questions"):
    # for question_type in ["formal", "conversational"]:
    for question_type in ["conversational"]:
        question = row[f"question_{question_type}"]
        embedding = row[f"question_{question_type}_embedding"]
        if np.isnan(row['chunck']):
          true_idx = 0
        else:
          true_idx = int(row['chunck'])
        # Retrieve top 20 sentences
        top_20, rank = retrieve_top_sentences(embedding, df_documents['embedding'].tolist(), true_idx, 20)
        context_r1 = format_prompt(question, top_20['sentence'].iloc[:10].tolist())
        # context_r2 = format_prompt(question, top_20['sentence'].iloc[10:20].tolist())

        # Calculate rank and retrieval status
        df_questions.loc[idx, f"{question_type}_rank"] = rank
        df_questions.loc[idx, f"{question_type}_retrieved"] = 1 if rank <= 10 else 0.5 if rank <= 20 else 0
        # Get answers using both model versions
        model = f"meta/meta-llama-3-70b-instruct"
        answer_r1 = get_answer(model, context_r1)
        # # answer_r2 = "None" if "لا ادري" in answer_r1 else get_answer(model, context_r2)

        # # Log answers and labels
        df_questions.loc[idx, f"{question_type}_answer_r1"] = answer_r1
        # # df_questions.loc[idx, f"{model_size}_{question_type}_answer_r2"] = answer_r2

Processing Questions:   0%|          | 0/150 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sentence_transformers/util.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  a = torch.tensor(a)
Processing Questions:   5%|▍         | 7/150 [00:16<05:26,  2.28s/it]

Attempt 1 failed: 


Processing Questions:  68%|██████▊   | 102/150 [07:39<03:30,  4.39s/it]

Attempt 1 failed: 


Processing Questions: 100%|██████████| 150/150 [10:28<00:00,  4.19s/it]


In [ ]:
column_order = [
    'question_formal', 'question_conversational', 'context', 'answer',
     'conversational_retrieved', 'conversational_answer_r1', 'conversational_rank',
    'page', 'chunck', 'question_formal_embedding', 'question_conversational_embedding'
]
df_questions = df_questions[column_order]
df_questions.to_excel("cleaned_llama_results2.xlsx", index=False)

In [ ]:
# column_order = [
#     'question_formal', 'question_conversational', 'context', 'answer',
#     'formal_rank', 'formal_retrieved',
#     '70b_formal_answer_r1', '70b_formal_answer_r2',
#     '8b_formal_answer_r1', '8b_formal_answer_r2',
#     'conversational_rank', 'conversational_retrieved',
#     '70b_conversational_answer_r1', '70b_conversational_answer_r2',
#     '8b_conversational_answer_r1', '8b_conversational_answer_r2',
#     'page', 'chunck', 'question_formal_embedding', 'question_conversational_embedding'
# ]

column_order = [
    'question_formal', 'question_conversational', 'context', 'answer',
     'formal_retrieved', 'formal_answer_r1', 'formal_rank',
    'page', 'chunck', 'question_formal_embedding', 'question_conversational_embedding'
]
df_questions = df_questions[column_order]

In [ ]:
df_questions.to_csv("cleaned_dataset.csv", index=False)

In [ ]:
df_questions.to_excel("cleaned_dataset.xlsx", index=False)

# Improve text quality

In [ ]:
system_prompt = """You are a skilled Arabic language expert and text editor. Your task is to refine and enhance the quality of an Arabic text chunk extracted from a document and had errors due to bad parsing. Focus on these key areas:

1. Spelling and Grammar Correction:
    * Carefully examine the text for any spelling mistakes or grammatical errors.
    * Correct errors using appropriate Arabic grammar and spelling rules.

2. Coherence and Fluency:
    * Ensure the sentences within the chunk flow smoothly and logically.
    * Adjust sentence structure or word choice to improve readability.
    * Remove unnecessary repetitions or redundant information.

3. Clarity and Conciseness:
    * Make sure the text clearly conveys its intended meaning.
    * Eliminate any ambiguity, vagueness, or text that seems random due to bad parsing.

Additional Instructions:
* Preserve the original meaning of the text while making corrections.
* Avoid over-editing or changing the intended meaning.
* Prioritize accuracy and clarity over stylistic embellishments.

Focus on ensuring logical flow of information especially with numbers as they are usually out of place.
Output the refined text after "Revised:". Then any additional info after "Note:"
"""


def get_edit(prompt):
    try:
        response = replicate.run(
    "meta/meta-llama-3-8b-instruct",
    input={
        "prompt": prompt,
        "system_prompt": system_prompt,
    },
)
        return "".join(response)
    except Exception as e:
        print(e)
        return "NA"

In [ ]:

for idx, row in tqdm(df_documents.iterrows(), total=df_documents.shape[0], desc="Processing Sentences"):
  sentence = row["sentence"]
  c = 0
  while c < 5:
    c += 1
    answer = get_edit(sentence)
    pattern = r"Revised:(.*?)Note:"
    match = re.search(pattern, answer, re.S)
    if match:
        text = match.group(1).strip()  # strip() removes any extra whitespace
        c = 10
    if c == 5:
      print(f"Err: {idx}")
      break
  df_documents.loc[idx, "refined_sentence"] = text

Processing Sentences: 100%|██████████| 514/514 [31:02<00:00,  3.62s/it]

Err: 513


In [ ]:
df_documents

,sentence,embedding,refined_sentence
0,بيـــان\nالميزانية العامة للدولة\n)م2024( هـ14...,"[0.024268262088298798, 0.008252768777310848, 0...",بيان الميزانية العامة للدولة\nم 2024 هـ 1446 -...
1,تطورات الاقتصاد العالمي\n15ب.,"[0.008082271553575993, 0.014534637331962585, -...",تطورات الاقتصاد العالمي
2,تطورات الاقتصاد المحلي\n27\n:القسم الثاني\nم...,"[0.03123318962752819, -0.005314670968800783, -...",تطورات الاقتصاد المحلي\n:القسم الثاني\nمواليد2...
3,تطورات أداء المالية العامة في العام\n36م و...,"[0.013291748240590096, 0.008054817095398903, -...",تطورات أداء المالية العامة في العام 2016 وتقدي...
4,ميزانية عام\n43م على مستوى القطاعات2024 ج.,"[-0.005651162937283516, 0.0024492705706506968,...",ميزانية عام 2024 على مستوى القطاعات 43 مليار.
...,...,...,...
509,العوامل الإيجابية\nعلــى الرغـم مـن اسـتمرار ح...,"[0.013357712887227535, -0.0016816002316772938,...",العوامل الإيجابية\nعلى الرغم من استمرار حالة ع...
510,ممــا يحفــز أســواق الاســتثمار ويحافــظ \nعل...,"[0.00914826337248087, -0.0192567128688097, -0....",ما يُحفز أسواق الاستثمار ويحافظ على معادلات ال...
511,\nم علــى الموازنــة بيــن الحفــاظ علــى اســ...,"[3.37085803039372e-05, -0.0014684725319966674,...",يتعاظم واهمية التوازن بين الحفاظ على استدامة ا...
512,وعمـل صنـدوق الاســتثمارات العامـة 2023\n علــ...,"[0.016185462474822998, -0.0026623045559972525,...",وعمل صندوق الاستثمارات العامة 2023 على الاستثم...


In [ ]:
df_documents.to_csv("document_dataset_refined.csv", index=False)

In [ ]:
import re

def extract_chunk(text):
    # Define the pattern to find the text between "Revised:" and "Note:"
    pattern = r"Revised:(.*?)Note:"
    match = re.search(pattern, text, re.S)
    if match:
        return match.group(1).strip()  # strip() removes any extra whitespace
    else:
        print("wrong")

# Example usage
print(extract_chunk(answer))  # This should print the text between "Revised:" and "Note:"

# Example when the keywords are missing or not in order
text2 = "This document lacks proper markers. Note: Here's a note without a revision."
extract_chunk(text2)  # This should print "wrong"


بيان الميزانية العامة للدولة
م2024 هـ1446 - 1445 للعام المالي

مقدمة
الفهرس

الملخص التنفيذي لإلطار المالي والاقتصادي للميزانية

القسم الأول: التطورات والآفاق الاقتصادية لعام 2024
م ومدى المتوسط 2024
wrong
